## Carregando dados de um arquivo para um Dataframe

 * Vamos analisar dados sobre quilometragem em bikes na India, buscando realizar:
 
 1. Entender as cidades que mais possuem quilômetros rodados
 2. Enterder as cidades com as médias maiores  (ou a maior)
 3. Entender as cidades com as máximas maiores (ou a maior)
 4. Entender as cidades com as minimas menores (ou a maior)
 5. Verificar qual cidade tem mais registros e analisá-los
   - Analisar a média, máximo e mínimo desta cidade
   - Analisar pelo campo que descreve as bikes
   - Analisar se as bikes na maioria são de primeiro, segundo donos.
   - Gerando gráficos sobre os dados
 
 

In [1]:
#importando sparksession
from pyspark.sql import SparkSession

#criando um objeto sparksession object e um appName 
sparkSession=SparkSession.builder.appName("bikes").getOrCreate()

In [2]:
# Fazendo a leitura do arquivo Bikes.csv
bikes = sparkSession.read.option("header", "true").csv("/Users/claud/Desktop/EngUd/pyspark/dados/Bikes.csv")

In [3]:
bikes.show(5)

+--------------------+--------+---------+----------+-----------+---+-----+-------------+
|           bike_name|   price|     city|kms_driven|      owner|age|power|        brand|
+--------------------+--------+---------+----------+-----------+---+-----+-------------+
|TVS Star City Plu...| 35000.0|Ahmedabad|   17654.0|First Owner|3.0|110.0|          TVS|
|Royal Enfield Cla...|119900.0|    Delhi|   11000.0|First Owner|4.0|350.0|Royal Enfield|
|Triumph Daytona 675R|600000.0|    Delhi|     110.0|First Owner|8.0|675.0|      Triumph|
|TVS Apache RTR 180cc| 65000.0|Bangalore|   16329.0|First Owner|4.0|180.0|          TVS|
|Yamaha FZ S V 2.0...| 80000.0|Bangalore|   10000.0|First Owner|3.0|150.0|       Yamaha|
+--------------------+--------+---------+----------+-----------+---+-----+-------------+
only showing top 5 rows



In [4]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL
bikes.createOrReplaceTempView("tab_bikes")
sparkSession.sql("select * from tab_bikes").show()

+--------------------+--------+---------+----------+------------+---+-----+-------------+
|           bike_name|   price|     city|kms_driven|       owner|age|power|        brand|
+--------------------+--------+---------+----------+------------+---+-----+-------------+
|TVS Star City Plu...| 35000.0|Ahmedabad|   17654.0| First Owner|3.0|110.0|          TVS|
|Royal Enfield Cla...|119900.0|    Delhi|   11000.0| First Owner|4.0|350.0|Royal Enfield|
|Triumph Daytona 675R|600000.0|    Delhi|     110.0| First Owner|8.0|675.0|      Triumph|
|TVS Apache RTR 180cc| 65000.0|Bangalore|   16329.0| First Owner|4.0|180.0|          TVS|
|Yamaha FZ S V 2.0...| 80000.0|Bangalore|   10000.0| First Owner|3.0|150.0|       Yamaha|
|    Yamaha FZs 150cc| 53499.0|    Delhi|   25000.0| First Owner|6.0|150.0|       Yamaha|
|Honda CB Hornet 1...| 85000.0|    Delhi|    8200.0| First Owner|3.0|160.0|        Honda|
|Hero Splendor Plu...| 45000.0|    Delhi|   12645.0| First Owner|3.0|100.0|         Hero|
|Royal Enf

In [ ]:
# Selecionando dados para analisar a maior média de quilometros percorridos
sparkSession.sql("select city, avg(kms_driven) as media_km from tab_bikes group by city order by media_km desc ").show(1)

In [ ]:
# Selecionando dados para analisar a maior quilometragem atingida
sparkSession.sql("select city, max(kms_driven) as maximo_km from tab_bikes group by city order by maximo_km desc").show(1)

In [ ]:
# Selecionando dados para analisar a menor quilometragem atingida
sparkSession.sql("select city, min(kms_driven) as minimo_km from tab_bikes group by city order by minimo_km asc").show(1)

In [ ]:
# # Selecionando dados para analisar a média de potencia das bicicletas e o total por cidade
sparkSession.sql("select brand, avg(power) as media_potencia, count(*) as total from tab_bikes group by brand order by total desc").show()

In [ ]:
# Selecionando dados para analisar a cidade de Bajaj que tem a maior quantidade de registros, colocando em um Dataframe
bajaj=sparkSession.sql("select * from tab_bikes where brand='Bajaj'")

In [ ]:
# Criando uma tabela temporária apenas com os dados de Bajaj
bajaj.createOrReplaceTempView("tab_bikes2")

In [ ]:
# Identifiando o tipo de objeto
type(bajaj)

In [ ]:
# Selecionando os dados da cidade de Bajaj
sparkSession.sql("select count(*) from tab_bikes2").show()

In [ ]:
# Selecionando os dados da cidade de Bajaj, entendo o valor médio,máximo e mínimo 
sparkSession.sql("select avg(kms_driven) as media,max(kms_driven) as maximo,min(kms_driven) as minimo from tab_bikes").show()

In [ ]:
# Selecionando os dados da cidade de Bajaj, entendo o valor médio,máximo e mínimo por bike_name
sparkSession.sql("select bike_name, avg(kms_driven) as media,max(kms_driven) as maximo,min(kms_driven) as minimo from tab_bikes group by bike_name order by 1 asc").show(70)

In [ ]:
# Selecionando os dados da cidade de Bajaj, entendendo como foram adquiridas as Bikes
sparkSession.sql("select owner,count(*) as total from tab_bikes group by owner order by 2 desc").show()

In [ ]:
# Cria um Dataframe pandas para plotar os dados
bajaj_df=sparkSession.sql("select owner,count(*) as total from tab_bikes group by owner order by 2 desc")
bajaj_pandas= bajaj_df.toPandas()

In [ ]:
# utiliza a biblioteca matplotlib para criar um gráfico de barras para entender como são adquiridas as Bikes
import matplotlib.pyplot as plotar
bajaj_pandas.plot(kind='bar',x='owner',y='total')
plotar.title('Como foram adquiridas as Bikes')
plotar.xlabel('Por uso')
plotar.ylabel('Total proprietários')
plotar.show()